In [2]:
import torch

In [3]:
words = open("../names.txt", "r").read().splitlines()

In [4]:
words[:2]

['emma', 'olivia']

In [5]:
chars = sorted(list(set("".join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s,i in stoi.items()}

In [6]:
# create the dataset
xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2] + 27
    ix3 = stoi[ch3]
    xs.append((ix1,ix2))
    ys.append(ix3)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

number of examples:  392226


In [7]:
words[0]

'emma'

In [8]:
xs[:4]

tensor([[ 0, 32],
        [ 5, 40],
        [13, 40],
        [13, 28]])

In [9]:
ys[:4]

tensor([13, 13,  1,  0])

In [10]:
xs.shape

torch.Size([196113, 2])

In [11]:
import torch.nn.functional as F

In [12]:
# initialize the neural network
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator = g, requires_grad = True)

In [14]:
xenc = F.one_hot(xs, num_classes=54).float()

In [15]:
xenc.shape

torch.Size([196113, 2, 54])

In [19]:
# gradien descent
for k in range(100):
    
    # forward pass
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims = True)
    
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
    print(loss.item())
    
    # backward pass
    W.grad = None
    loss.backward()
    
    # update
    W.data += -50 * W.grad

IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [392226], [196113]